# BERT - Fine-Tuning for Movie Reviews
* Text Retrieval and Mining, BSc BAN, 2023-2024
* Author: [Julien Rossi](mailto:j.rossi@uva.nl)



**GPU Warning**

This notebook requires a GPU to run properly.

This notebook illustrates how we can use a model of the BERT family for a text classification task, using the huggingface library:
* Load the data into a `Dataset`-like class: the class `IMDbDataset`
  * Implements the operator `len` (method `__len__`) (remember Intro to Programming and dunder methods ?)
  * Implements the operator `[]` (method `__getitem__`)
  * Inherits from `torch.Dataset`
  * The text is tokenized with a specific `Tokenizer`, loaded from a repository
  * This tokenizer creates all the inputs for the BERT model
* Set up the training arguments
  * Parameters for the Deep Learning part of training
  * Object of class `TrainingArguments`
* Create a model ready for text classification
  * loads pre-trained weights from a repository (using the name `'distilbert-base-uncased'`)
  * The object of class `DistilBertForSequenceClassification` contains all the calculation graph from inputs to outputs
* Fine-Tune the model on a text classification task
  * Use the labeled data we have: text + label for training

___

This notebook shows how *easy* it is to summon a model with a few hundreds of millions of parameters and then fine-tune it on our specific classification task, thanks to [Huggingface](https://huggingface.co/docs/transformers/en/index).


In [ ]:
!pip install accelerate

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [ ]:
import requests

r = requests.get("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")
assert r.status_code == 200

In [ ]:
import io
import tarfile

with tarfile.open(fileobj=io.BytesIO(r.content), mode="r:gz") as tgz:
    tgz.extractall()

In [ ]:
import torch

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, DistilBertConfig
from transformers import Trainer, TrainingArguments

In [ ]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
MODEL = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:
!rm -rf ./results
!rm -rf ./logs

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=1e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    fp16=True,
    do_train=True,
    do_eval=True
)

model = DistilBertForSequenceClassification.from_pretrained(MODEL)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()